In [ ]:
import tqdm, re, pandas as pd, random
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
stopwords= set(STOPWORDS)
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
def get_memory():
    status= open('/proc/self/status').read()
    memuse= int(re.findall('VmRSS:\s*(\d+)\skB', status)[0]) / 1024 ** 2
    print('Resident memory: %.2f GB' % memuse)

In [ ]:
get_memory()

In [ ]:
def loadPickle(name):
    return pd.read_pickle(f'../data/{name}')
def loadTxt(name):
    return [i.strip() for i in open(f'../data/{name}', 'r').readlines()]
def dumpPickle(df, name):
    df.to_pickle(f'../data/{name}')

In [ ]:
RegB= loadPickle('Regular_Before_py37.pkl')
RegA= loadPickle('Regular_After_py37.pkl')
# Find the common users
regularBefore= RegB[RegB['user.id_str'].isin(RegA['user.id_str'])].copy()
regularAfter= RegA[RegA['user.id_str'].isin(RegB['user.id_str'])].copy()
# Set the type of user
regularBefore['typeOfUser']= 'regularBefore'
regularAfter['typeOfUser']= 'regularAfter'

In [ ]:
ResB= loadPickle('Restored_Before_py37.pkl')
ResA= loadPickle('Restored_After_py37.pkl')
# Find he common users
restoredBefore= ResB[ResB['user.id_str'].isin(ResA['user.id_str'])==True].copy()
restoredAfter= ResA[ResA['user.id_str'].isin(ResB['user.id_str'])==True].copy()
# Set the type of user
restoredBefore['typeOfUser']= 'restoredBefore'
restoredAfter['typeOfUser']= 'restoredAfter'

In [ ]:
pklDict= {
    'restoredBefore': restoredBefore,
    'restoredAfter': restoredAfter,
    'regularBefore': regularBefore,
    'regularAfter': regularAfter,
}
ResA= None
ResB= None
RegA= None
RegB= None
regularBefore= None
regularAfter= None
restoredBefore= None
restoredAfter= None

In [ ]:
def showTopics(data):
    # create dictionary
    splitDf= data.map(lambda x: [i.lower() for i in re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split() if len(i)>=3 and i not in stopwords])
    text_dict= Dictionary(splitDf)

    # Output the frequent topics LDA Model
    tweets_bow= [text_dict.doc2bow(tweet) for tweet in splitDf]
    tweets_lda= LdaModel(tweets_bow, num_topics= 5, id2word= text_dict, random_state= 1, passes= 10)
    for index, topic in tweets_lda.show_topics(formatted=False):
        print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))

In [ ]:
showTopics(pklDict['restoredBefore']['full_text'])

In [ ]:
showTopics(pklDict['restoredAfter']['full_text'])

In [ ]:
showTopics(pklDict['regularBefore']['full_text'])

In [ ]:
showTopics(pklDict['regularAfter']['full_text'])